<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_1_tuning_healthnews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [0]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    989      0 --:--:-- --:--:-- --:--:--   989
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [0]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-1.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [0]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-1.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heels hurt
2,4,Discoloration of skin [blotchy] on the neck
3,6,diastolic dysfunction
4,8,arthritis


In [0]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-1.test.csv",header=None)
test.head()

,0,1
0,2,heel pain
1,2,right heel became painful
2,8,Arthritis
3,8,arthristis
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_healthnews_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [0]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,6.953360,5.829993,0.141629,02:06


In [0]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.844808,5.318971,0.181362,02:06


In [0]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.425507,5.260486,0.186049,02:05


In [0]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.362593,5.198071,0.190904,02:05


In [0]:
learn.save_encoder('healthnews_fold1_first')

In [0]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('healthnews_fold1_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.533660,4.584394,0.290557,01:08


In [0]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.713555,4.196432,0.328087,01:17


In [0]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.402843,3.896454,0.364407,03:21


In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.235132,3.692521,0.381356,03:31
1,3.039960,3.589388,0.403148,03:17


In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.924152,3.509816,0.420097,03:21
1,2.765942,3.449742,0.428571,03:14


In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.664982,3.419598,0.429782,03:13
1,2.605366,3.339242,0.447942,03:24


In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.612668,3.320987,0.433414,03:19
1,2.465142,3.252347,0.456416,03:25


In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.487782,3.256766,0.444310,03:34
1,2.417330,3.199333,0.456416,03:22


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.347083,3.246412,0.463680,03:37
1,2.398868,3.177434,0.446731,03:14
2,2.243058,3.150736,0.466102,03:29
3,2.257035,3.179996,0.467312,03:26


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.265453,3.169123,0.464891,03:20
1,2.271064,3.194969,0.467312,03:33
2,2.216218,3.162269,0.476998,03:36
3,2.126547,3.149972,0.476998,03:22


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.062262,3.131138,0.462470,03:34
1,2.182803,3.195471,0.460048,03:21
2,2.098033,3.158900,0.474576,03:26
3,2.082375,3.187079,0.473366,03:22


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.075064,3.199009,0.467312,03:30
1,2.076108,3.178097,0.470944,03:20
2,2.073515,3.174212,0.478208,03:21
3,1.991656,3.201557,0.478208,03:09


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.035118,3.252703,0.468523,03:32
1,1.982438,3.152334,0.479419,03:16
2,1.945557,3.166575,0.481840,03:32
3,1.901466,3.154174,0.481840,03:34


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.924339,3.297805,0.473366,03:23
1,2.096794,3.237428,0.470944,03:24
2,1.895940,3.209881,0.478208,03:29
3,1.932285,3.237580,0.483051,03:15


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.992742,3.293792,0.481840,03:18
1,2.063722,3.310431,0.475787,03:30
2,1.967420,3.208848,0.480630,03:18
3,1.901426,3.254750,0.476998,03:28


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.822976,3.300332,0.474576,03:08
1,2.002398,3.247747,0.479419,03:12
2,1.973322,3.314237,0.483051,03:12
3,1.874344,3.239328,0.486683,03:23


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.950500,3.357494,0.473366,03:28
1,1.977379,3.277136,0.491525,03:16
2,1.948478,3.372787,0.484262,03:11
3,1.951412,3.274797,0.484262,03:23


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.906278,3.278147,0.483051,03:29
1,2.027251,3.329026,0.472155,03:31
2,1.914472,3.308940,0.480630,03:30
3,1.931333,3.396572,0.474576,03:18


In [0]:
# save the best model

learn.save_encoder('ask-1-tuning-healthnews')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [0]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

867
867
[2, 2, 8, 1026, 8, 8, 1026, 16, 427, 1026, 1026, 427, 27, 584, 1028, 487, 37, 37, 37, 37, 37, 37, 37, 704, 37, 228, 427, 1035, 44, 517, 301, 49, 301, 1028, 1028, 1028, 1028, 49, 443, 443, 1028, 226, 427, 61, 62, 1028, 77, 208, 1035, 1028, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 172, 1028, 1028, 1028, 1028, 1028, 289, 1028, 289, 80, 961, 80, 80, 80, 80, 865, 289, 443, 89, 1028, 92, 92, 92, 1026, 1026, 443, 443, 427, 736, 1028, 1028, 1028, 265, 1028, 1028, 648, 427, 100, 100, 100, 100, 337, 100, 100, 419, 107, 49, 172, 258, 103, 269, 105, 72, 72, 686, 289, 289, 699, 211, 111, 111, 111, 111, 111, 289, 100, 1028, 117, 445, 125, 125, 125, 1028, 1028, 125, 125, 125, 125, 125, 125, 125, 125, 125, 802, 654, 509, 226, 443, 1028, 1035, 473, 76, 443, 142, 643, 864, 289, 158, 443, 1028, 1028, 1028, 269, 741, 289, 190, 356, 289, 1028, 1028, 1028, 107, 745, 182, 334, 82, 182, 182, 182, 182, 18

In [0]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

238
0.27450980392156865
